# Pytorch深度学习：自然语言处理
**感谢原作者的教程，查看英文原版请点击[原文地址](https://github.com/spro/practical-pytorch)。**

这个课程会带你学习使用Pytorch做深度学习程序的关键思想。很多概念（比如“运算图”概念以及“自动求导”）不仅仅独属于Pytorch，也和很多深度学习工具包相关联。

作者写这个教程主要是为了那些从未利用深度学习框架（像Tensorflow, Theano, Keras, Dynet）写过任何代码的人群学习NLP。这里假设读者有核心NLP问题的知识：部分对话标记，语言模型等等。还假设读者通过一些介绍型的AI课程（像是Russel和Norvig写的人工智能一书）熟悉神经网络。通常来说，这些课程包括了基本的前馈神经网络的反向传播算法，而且指出了它们是线性或者非线性的链式组成成分。这个课程会帮助你开始学习深度学习代码，给你这方面的先决知识。

注意这大都是模型，而不是数据。对于所有的模型，作者仅仅写了一小部分的小维度测试样本，即可让读者看见当训练时权重时怎样发生变化的。如果读者有一些真实的数据想要去尝试，需要有能力从这个notebook中的模型删除掉些数据，然后再使用它。

In [1]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# 1. Torch张量库的介绍

所有深度学习都在张量上进行计算，张量就是一个可以被超过三个维度来索引的矩阵的一般概念。我们稍后会深入了解其准确的意义。首先，我们来看看我们可以通过张量做些什么。

### 创建张量(Tensors)
张量可以从Python列表中通过torch被创建。Tensor()函数。

In [2]:
# Create a torch.Tensor object with the given data.  It is a 1D vector
V_data = [1., 2., 3.]
V = torch.Tensor(V_data)
print(V)

# Creates a matrix
M_data = [[1., 2., 3.], [4., 5., 6]]
M = torch.Tensor(M_data)
print(M)

# Create a 3D tensor of size 2x2x2.
T_data = [[[1.,2.], [3.,4.]],
          [[5.,6.], [7.,8.]]]
T = torch.Tensor(T_data)
print(T)


 1
 2
 3
[torch.FloatTensor of size 3]


 1  2  3
 4  5  6
[torch.FloatTensor of size 2x3]


(0 ,.,.) = 
  1  2
  3  4

(1 ,.,.) = 
  5  6
  7  8
[torch.FloatTensor of size 2x2x2]



什么是一个3D张量？想象一下，如果你有一个向量，索引这个向量的话你会得到一个标量。如果你有一个矩阵，索引这个矩阵的话会得到一个向量。如果你有一个三维张量，然后索引它你会得到一个矩阵！

一个术语上的注解：当笔者在本教程提到“张量”的时候，它泛指torch.Tensor对象。矩阵和向量是一种特殊的torch.Tensors，其维度分别为1和2。当笔者讨论到三维张量的时候，他通常会用到术语“3D tensor"。

In [3]:
# Index into V and get a scalar
print(V[0])

# Index into M and get a vector
print(M[0])

# Index into T and get a matrix
print(T[0])

1.0

 1
 2
 3
[torch.FloatTensor of size 3]


 1  2
 3  4
[torch.FloatTensor of size 2x2]



读者还可以创建其他数据格式的张量。如你所见，默认值是浮点型（Float)。创建整型张量，可以尝试torch.LongTensor()。可以通过检阅使用文档获取更多数据类型，但是Float和Long是最常见的。

读者可以使用torch.randn()函数，通过随机数据和提供指定维度来创建张量。

In [4]:
x = torch.randn((3, 4, 5))
print(x)


(0 ,.,.) = 
 -0.3087 -0.6728  1.0570  0.5169 -0.5722
  0.3205  0.4440  0.3725 -1.5178 -1.1531
  2.4737 -0.1481  0.6417  0.2472 -1.7715
  1.8946 -0.5377 -1.2909 -0.8704 -0.4269

(1 ,.,.) = 
 -0.4384 -0.7725  0.1785 -0.9593 -0.3452
  0.3321 -0.1275  0.4012  0.1103  1.7281
 -1.1975  1.2458  1.7392 -0.6578  0.2765
 -0.5161  1.2120 -1.2803  0.1048  0.0766

(2 ,.,.) = 
 -1.2658  0.2734  1.0934 -0.3089 -0.9949
  1.9348 -0.5163  2.1339  0.6003 -0.1163
 -0.2555 -0.1470 -0.5322  0.9048  1.4007
  0.6043  0.6713  0.6780  0.3931 -0.5096
[torch.FloatTensor of size 3x4x5]



### 操作张量
读者可以很多种期望的方式来操作张量

In [5]:
x = torch.Tensor([ 1., 2., 3. ])
y = torch.Tensor([ 4., 5., 6. ])
z = x + y
print(z)


 5
 7
 9
[torch.FloatTensor of size 3]



阅读[使用文档](http://pytorch.org/docs/torch.html)查询完整列表，包括大量可行的操作方法，它们拓展到了不仅仅只有数学的操作方法。待会我们会用到的一个有用的操作是串联（concat）。

In [6]:
# By default, it concatenates along the first axis (concatenates rows)
x_1 = torch.randn(2, 5)
y_1 = torch.randn(3, 5)
z_1 =torch.cat([x_1, y_1])
print(z_1)

# Concatenate columns:
x_2 = torch.randn(2, 3)
y_2 = torch.randn(2, 5)
z_2 = torch.cat([x_2, y_2], 1) # second arg specifies which axis to concat along
print(z_2)

# If your tensors are not compatible, torch will complain.  Uncomment to see the error
# torch.cat([x_1, x_2])


 0.6930  0.0136  0.8145  0.6392 -0.1040
 1.0693  0.7222 -0.9371 -0.2101  1.8514
 0.4354  0.3018  0.7542  0.3365  0.0728
 0.4461 -0.8490 -0.2936  0.9696  0.0905
-0.1991 -0.5358 -0.6633 -0.1786 -2.5800
[torch.FloatTensor of size 5x5]


 0.5264 -1.4923 -1.4876  0.1094  0.7057 -0.0816  0.0974 -0.6664
-0.6045  2.0085 -0.2481 -0.2512  0.4174  1.4055 -0.5064 -0.8408
[torch.FloatTensor of size 2x8]



### 重塑张量
通过 .view() 方法可以去重塑一个张量的形状。这个方法有着重要的作用，因为非常多的神经网络运算会需要输入向量是一个特定形状的向量。读者会经常需要在将数据放入组件之前进行重塑。

In [7]:
x = torch.randn(2, 3, 4)
print(x)
print(x.view(2, 12)) # Reshape to 2 rows, 12 columns
print(x.view(2, -1)) # Same as above.  If one of the dimensions is -1, its size can be inferred


(0 ,.,.) = 
 -0.4279  1.2043  0.2500 -0.0412
 -0.3051  0.7754 -0.5304 -0.1212
 -1.5656  0.3708 -0.1095  0.2443

(1 ,.,.) = 
  1.7076 -1.4903 -0.4272  0.2099
  0.5465  1.4020  1.6843  0.2993
  0.1312  0.1642  1.0890  0.0695
[torch.FloatTensor of size 2x3x4]



Columns 0 to 9 
-0.4279  1.2043  0.2500 -0.0412 -0.3051  0.7754 -0.5304 -0.1212 -1.5656  0.3708
 1.7076 -1.4903 -0.4272  0.2099  0.5465  1.4020  1.6843  0.2993  0.1312  0.1642

Columns 10 to 11 
-0.1095  0.2443
 1.0890  0.0695
[torch.FloatTensor of size 2x12]



Columns 0 to 9 
-0.4279  1.2043  0.2500 -0.0412 -0.3051  0.7754 -0.5304 -0.1212 -1.5656  0.3708
 1.7076 -1.4903 -0.4272  0.2099  0.5465  1.4020  1.6843  0.2993  0.1312  0.1642

Columns 10 to 11 
-0.1095  0.2443
 1.0890  0.0695
[torch.FloatTensor of size 2x12]



# 2. 运算图与自动微分
对于一个有效的深度学习程序来说”运算图“是非常重要的一个概念。因为它可以让读者省去自己去写反向传播梯度。一个运算图仅仅是一个说明书用于表达你怎么样将数据关联至输出的。因为”图“完全指定好了哪个参数会被涉及到哪种运算，所以它包含了足够的信息量去计算导数。这听起来可能有点模糊，所以让我们看看怎么样使用Pytorch的一个基础类：autograd.Variable

首先，从一个程序员的角度来想。什么被存储在刚才我们创建的torch.Tensor对象？显然是数据和形状，可能还有一些别的玩意儿。但是当我们添加两个张量到一起，我们获得了一个输出张量。这个输出张量只知道其数据和形状，它并不知道它是通过两个别的张量合成而来（它可以从一个文件中被加载而来，也可以通过一些别的操作而来，等等）

但是Variable类保留了自身被创建的过程。让我们从操作中了解一下。

In [8]:
# Variables wrap tensor objects
x = autograd.Variable( torch.Tensor([1., 2., 3]), requires_grad=True )
# You can access the data with the .data attribute
print(x.data)

# You can also do all the same operations you did with tensors with Variables.
y = autograd.Variable( torch.Tensor([4., 5., 6]), requires_grad=True )
z =(x + y)
print(z)

# BUT z knows something extra.
print(z.creator)
print(x.creator)


 1
 2
 3
[torch.FloatTensor of size 3]

Variable containing:
 5
 7
 9
[torch.FloatTensor of size 3]

None


所以Variables对象知道是什么创建的它们。z知道它并不是通过一个文件被读取，也不是通过乘法、指数运算或者别的什么创建的。如果读者保持关注z.creator,会自行找到x和y。

但是它是怎么帮助我们运算梯度呢？

In [9]:
# Lets sum up all the entries in z
s = z.sum()
print(s)
print(s.creator)

Variable containing:
 21
[torch.FloatTensor of size 1]



那么现在，对于第一个成分x什么才是这组求和过程的导数？在数学中，我们表达为：
$$ \frac{\partial s}{\partial x_0} $$
然后s知道了它是被张量z的求和过程产生的，z知道它是被x+y的求和产生的，所以有：
$$ s = \overbrace{x_0 + y_0}^\text{$z_0$} + \overbrace{x_1 + y_1}^\text{$z_1$} + \overbrace{x_2 + y_2}^\text{$z_2$} $$
所以s有了足够的信息量去确定我们想要的导数是1！

当然这降低了如何去实际计算导数的挑战。这里的关键是s沿途带着足够的信息，所以可能去计算导数。实际上，Pytorch的开发者编写了让sum()和+操作符去了解怎么去计算梯度，然后运行反向传播算法。关于这个算法深入的讨论超出了这个教程的范畴。

接下来让Pytroch计算梯度来证明刚才推断的正确：（注意如果读者将这个区块多次运行的话，梯度会加大。这是因为Pytorch累积了梯度导 .grad属性当中，这对很多模型来说非常方便）

In [10]:
s.backward() # calling .backward() on any variable will run backprop, starting from it.
print(x.grad)

Variable containing:
 1
 1
 1
[torch.FloatTensor of size 3]



对于成为一个成功的深度学习程序员而言，了解下面模块是如何运行非常重要。

In [11]:
x = torch.randn((2,2))
y = torch.randn((2,2))
z = x + y # These are Tensor types, and backprop would not be possible

var_x = autograd.Variable( x )
var_y = autograd.Variable( y )
var_z = var_x + var_y # var_z contains enough information to compute gradients, as we saw above
print(var_z.creator)

var_z_data = var_z.data # Get the wrapped Tensor object out of var_z...
new_var_z = autograd.Variable( var_z_data ) # Re-wrap the tensor in a new variable

# ... does new_var_z have information to backprop to x and y?
# NO!
print(new_var_z.creator)
# And how could it?  We yanked the tensor out of var_z (that is what var_z.data is).  This tensor
# doesn't know anything about how it was computed.  We pass it into new_var_z, and this is all the information
# new_var_z gets.  If var_z_data doesn't know how it was computed, theres no way new_var_z will.
# In essence, we have broken the variable away from its past history

None


这对通过autograd.Variables计算而言是一个非常重要的规则（注意这是一个更加普遍的而不仅仅指在Pytorch里，在所有主流深度学习工具箱中都有这样一个相同的对象）：

**如果想通过损失函数的误差去对神经网络中的一部分进行反向传播运算，一定不可以破坏该变量（Variable)直至损失变量的整个传递过程。如果不小心破坏了，损失将对网络中的成分失去联系，那么参数将不能被更新。**

作者将这段粗体提示，是因为这个错误会通过非常微妙的方式令人难受（将会在下面展示一些类似方式），且不会导致代码崩溃或者报错，所以你必须非常小心。

# 3. 深度学习构建模块：线性变换，非线性变换以及目标

深度学习包括通过巧妙的方式将线性与非线性结合起来。非线性变换的介绍涉及到了强力模型。在这个区域，我们会运用这些关键模块，组成一个对象函数，然后来观察这个模型是如何训练的。

### 线性变换
深度学习一个核心的重要工作就是线性变换(affine map/ 仿射变换)，其代表一个函数$f(x)$包括一个矩阵$A$和向量$x,b$
$$f(x)=Ax+b$$
这里可以被学习的参数是$A$和$b$。通常来说，$b$被称作偏斜项(bias term)。

Pytorch和许多别的深度学习框架与传统的线性代数有一点不太一样。它映射”行“作为输入而不是”列“。这即是说，下面第$i$列的输出是输入$A$的第$i$列的映射加上偏移项。如下为例

In [12]:
lin = nn.Linear(5, 3) # maps from R^5 to R^3, parameters A, b
data = autograd.Variable( torch.randn(2, 5) ) # data is 2x5.  A maps from 5 to 3... can we map "data" under A?
print(lin(data)) # yes

Variable containing:
 0.7462 -0.6956 -0.6966
-1.2308  1.0765 -0.2363
[torch.FloatTensor of size 2x3]



### 非线性变换

首先，注意以下事实，它会解释为什么我们需要在第一个地方使用非线性运算。假设我们有两个线性变换$f(x)=Ax+b$和$g(x)=Cx+d$，那么$f(g(x))$会是什么样呢？
$$f(g(x))=A(Cx+d)+b=ACx+(Ad+b)$$
$AC$是一个矩阵而$Ad+b$是一个向量，所以我们看到包含多个线性变换其实是一个线性变换。

通过这个可以发现，如果想神经网络通过包含线性变换成为一个长链，这并不会对模型添加什么与简单的线性变换不一样的东西。

如果我们介绍夹在线性变换层中的非线性变换，这就不一样了，而且我们能因此构建更加强力的模型。

这有一些核心的非线性变换，$tanh(x),\sigma (x),ReLu(x)$都是非常常用。读者也许会好奇：“为什么是这些函数？我可以想到大量的其它非线性变换函数”。原因在于它们拥有易于运算的梯度，计算梯度对学习过程来说特别重要。举个栗子：
$$\frac{d\sigma}{dx}=\sigma(x)(1-\sigma(x))$$

一条简短的注释:也许读者通过一些介绍AI的课程中学习到很多神经网络的默认非线性变换是$\sigma(x)$，代表性地人们在实践中会回避去使用它。这是因为当变量绝对值增大的时候梯度会消逝地非常快。小的梯度意味着很难去训练，大部分人默认去用双曲正切函数或者是ReLU。

In [13]:
# In pytorch, most non-linearities are in torch.functional (we have it imported as F)
# Note that non-linearites typically don't have parameters like affine maps do.
# That is, they don't have weights that are updated during training.
data = autograd.Variable( torch.randn(2, 2) )
print(data)
print(F.relu(data))

Variable containing:
 0.2079 -1.4241
-0.8359  0.9315
[torch.FloatTensor of size 2x2]

Variable containing:
 0.2079  0.0000
 0.0000  0.9315
[torch.FloatTensor of size 2x2]



### Softmax和概率
函数$\rm{Softmax}(x)$也是一个非线性映射，但是它比较特别，通常用于一个网络的最后一次运算。这是因为它输入一个实数向量然后返回一个概率分布。它的定义如下：令$x$为一个实数向量（正数或负数都没限制），然后其第$i$个$\rm{Softmax}(x)$元素是
$$\frac{\rm{exp}(x_i)}{\Sigma_j \rm{exp}(x_j)}$$
这应该很明显输出是一个概率分布：每个元素都是非负的且全部元素的和为1.

读者也可以认为它是做了一个元素上的指数运算，令输入都变成非负的，然后除以一个标准化的常数。

In [14]:
# Softmax is also in torch.functional
data = autograd.Variable( torch.randn(5) )
print(data)
print(F.softmax(data))
print(F.softmax(data).sum()) # Sums to 1 because it is a distribution!
print(F.log_softmax(data)) # theres also log_softmax

Variable containing:
-0.3503
-0.2554
 3.0241
 0.4342
 0.8046
[torch.FloatTensor of size 5]

Variable containing:
 0.0273
 0.0300
 0.7964
 0.0598
 0.0865
[torch.FloatTensor of size 5]

Variable containing:
 1
[torch.FloatTensor of size 1]

Variable containing:
-3.6020
-3.5071
-0.2276
-2.8175
-2.4471
[torch.FloatTensor of size 5]



### 目标函数
目标函数是一个神经网络被训练去最小化的函数（在有些案例中也被称作损失函数或者代价函数）。它会被第一次训练实例时运行，运行整个神经网络，然后计算输出的损失。然后模型的参数会通过计算损失函数的导数进行更新。直觉上来说，如果模型完全确信其答案，但是答案确实错的，那么损失会非常高。如果完全确信其答案，且答案是正确的，那么损失会很低。

在训练样本上最小化代价函数的想法是指希望网络会如期地形成良好的概念，且在未见到的开发集(dev set)、测试集(test set)或生产环境样本上有较小的损失。负对数似然损失的样本损失函数，是非常常见的多任务分类目标。为了监督多任务分类，这表示通过最小化正确输出的负对数概率（或者等价地去最大化正确输出的对数似然概率）去训练网络。

# 4. 最优化和训练
所以我们能怎样为一个实例计算损失函数？我们要为它做些什么？我们看见早前的autograd.Variable知道如何考虑被计算的对象去计算梯度。既然我们的损失也是一个autograd.Variable对象，我们可以通过考虑全部用于计算的参数去计算其梯度！然后我们可以执行标准的梯度更新。令$\theta$是我们的参数，$L(\theta)$是损失函数，和$\eta$是一个正数的学习率。然后：
$$ \theta^{(t+1)} = \theta^{(t)} - \eta \nabla_\theta L(\theta) $$
在尝试去超越普通的梯度更新方法上有着非常大量的算法以及表现研究。很多尝试是基于训练次数去改变学习率。在真的十分感兴趣前，读者不必担心这些算法有些什么特别。Torch在torch.optim包里提供了许多方法，而且它们也是完全可视的。使用最简单的梯度更新是和一些复杂的算法一样的。尝试不同的更新算法和不同的更新参数（像不同的初始学习率）对优化网络的表现是比较重要的。通常，用诸如Adam或者RMSProp替换vanilla随机梯度下降（SGD）会显著促进表现。

# 5. 用Pytorch创建神经网络模块

在我们专注于NLP之前，让我们仅使用线性变换和非线性变换做一个用Pytorch构建网络的释例。我们会看到如何通过Pytorch构建负对数似然函数来计算一个损失函数，还有通过反向传播法进行参数更新。

所有网络模块应该从nn.Module继承以及重写forward()方法。就这个样本来看，继承自nn.Module向模块提供了功能。举个例子，它保留了其可训练参数的轨迹，你可以在CPU和GPU中用 .cude()或 .cpu()方法进行切换，等等。

让我们写一个输入一个稀疏的词袋表示然后输出双标签（分别是“English”和"Spanish"）的概率分布的神经网络释例。这个模型是一个逻辑回归。

### 示例：逻辑回归词袋分类
我们的模型会映射一个稀疏的词袋(BOW)表征至带标签的对数概率。我们会指定给每个单词在词典中一个索引。举个例子，我们的整个词库包括两个词"hello"和"world"，分别是0和1的指引。那么对于句子"hello hello hello hello"的词袋向量就是
$$ \left[ 4, 0 \right] $$
对于"hello world world hello"，就是
$$ \left[ 2, 2 \right] $$
等等。
一般而言，就是
$$ \left[ \text{Count}(\text{hello}), \text{Count}(\text{world}) \right] $$

将词袋向量表示为$x$，那神经网络的输出为
$$ \log \text{Softmax}(Ax + b) $$
这就是我们将输入传至线性变换然后做对数Softmax。

In [15]:
data = [ ("me gusta comer en la cafeteria".split(), "SPANISH"),
         ("Give it to me".split(), "ENGLISH"),
         ("No creo que sea una buena idea".split(), "SPANISH"),
         ("No it is not a good idea to get lost at sea".split(), "ENGLISH") ]

test_data = [ ("Yo creo que si".split(), "SPANISH"),
              ("it is lost on me".split(), "ENGLISH")]

# word_to_ix maps each word in the vocab to a unique integer, which will be its
# index into the Bag of words vector
word_to_ix = {}
for sent, _ in data + test_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
print(word_to_ix)

VOCAB_SIZE = len(word_to_ix)
NUM_LABELS = 2

{'me': 0, 'gusta': 1, 'comer': 2, 'en': 3, 'la': 4, 'cafeteria': 5, 'Give': 6, 'it': 7, 'to': 8, 'No': 9, 'creo': 10, 'que': 11, 'sea': 12, 'una': 13, 'buena': 14, 'idea': 15, 'is': 16, 'not': 17, 'a': 18, 'good': 19, 'get': 20, 'lost': 21, 'at': 22, 'Yo': 23, 'si': 24, 'on': 25}


In [16]:
class BoWClassifier(nn.Module): # inheriting from nn.Module!
    
    def __init__(self, num_labels, vocab_size):
        # calls the init function of nn.Module.  Dont get confused by syntax,
        # just always do it in an nn.Module
        super(BoWClassifier, self).__init__()
        
        # Define the parameters that you will need.  In this case, we need A and b,
        # the parameters of the affine mapping.
        # Torch defines nn.Linear(), which provides the affine map.
        # Make sure you understand why the input dimension is vocab_size
        # and the output is num_labels!
        self.linear = nn.Linear(vocab_size, num_labels)
        
        # NOTE! The non-linearity log softmax does not have parameters! So we don't need
        # to worry about that here
        
    def forward(self, bow_vec):
        # Pass the input through the linear layer,
        # then pass that through log_softmax.
        # Many non-linearities and other functions are in torch.nn.functional
        return F.log_softmax(self.linear(bow_vec))

In [17]:
def make_bow_vector(sentence, word_to_ix):
    vec = torch.zeros(len(word_to_ix))
    for word in sentence:
        vec[word_to_ix[word]] += 1
    return vec.view(1, -1)

def make_target(label, label_to_ix):
    return torch.LongTensor([label_to_ix[label]])

In [18]:
model = BoWClassifier(NUM_LABELS, VOCAB_SIZE)

# the model knows its parameters.  The first output below is A, the second is b.
# Whenever you assign a component to a class variable in the __init__ function of a module,
# which was done with the line
# self.linear = nn.Linear(...)
# Then through some Python magic from the Pytorch devs, your module (in this case, BoWClassifier)
# will store knowledge of the nn.Linear's parameters
for param in model.parameters():
    print(param)

Parameter containing:

Columns 0 to 9 
-0.1198  0.1166  0.0304  0.0985  0.1407 -0.1727 -0.1069  0.1455 -0.1883 -0.1737
 0.0351  0.1395 -0.0615  0.1733  0.1410 -0.0160  0.0775  0.1876  0.0301 -0.1139

Columns 10 to 19 
 0.1282  0.1252  0.0032  0.0923 -0.1159 -0.0464 -0.1952 -0.0433  0.1537  0.0162
-0.0582  0.0060  0.0491 -0.0424  0.1727  0.1215 -0.1013 -0.1034  0.1517  0.1936

Columns 20 to 25 
-0.1260 -0.0015 -0.0245 -0.0376  0.0599 -0.0127
 0.1120 -0.0218 -0.1036  0.1641  0.0404  0.1080
[torch.FloatTensor of size 2x26]

Parameter containing:
-0.0606
-0.1388
[torch.FloatTensor of size 2]



In [19]:
# To run the model, pass in a BoW vector, but wrapped in an autograd.Variable
sample = data[0]
bow_vector = make_bow_vector(sample[0], word_to_ix)
log_probs = model(autograd.Variable(bow_vector))
print(log_probs)

Variable containing:
-0.8200 -0.5806
[torch.FloatTensor of size 1x2]



上面的取值哪个反应了ENGLISH或SPANISH分类的对数概率？我们从未定义它，我们需要去训练它

In [20]:
label_to_ix = { "SPANISH": 0, "ENGLISH": 1 }

现在开始训练！我们需要传递一个实例去获取对数概率，计算损失函数，计算损失函数的梯度，然后再用梯度进行参数更新。损失函数会被Torch在nn package里提供，nn.NLLLoss()是我们想要的负对数似然函数损失。同时可以在toch.optim里定义优化方法，这里我们使用随机梯度下降法。

注意到NLLLoss的输入是一个包括对数概率的向量，以及一个目标标签。它不会为我们计算对数概率，这就是为什么这里的神经网络最后一层是对数softmax。如果不是对数softmax，那么损失函数nn.CrossEntropyLoss()和NLLLoss()一致。

In [21]:
# Run on test data before we train, just to see a before-and-after
for instance, label in test_data:
    bow_vec = autograd.Variable(make_bow_vector(instance, word_to_ix))
    log_probs = model(bow_vec)
    print(log_probs)
print(model.parameters())
print(next(model.parameters())[:,word_to_ix["creo"]]) # Print the matrix column corresponding to "creo"

Variable containing:
-0.5975 -0.7989
[torch.FloatTensor of size 1x2]

Variable containing:
-0.8619 -0.5488
[torch.FloatTensor of size 1x2]

<generator object Module.parameters at 0x7f0e6829de60>
Variable containing:
 0.1282
-0.0582
[torch.FloatTensor of size 2]



In [22]:
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# Usually you want to pass over the training data several times.
# 100 is much bigger than on a real data set, but real datasets have more than
# two instances.  Usually, somewhere between 5 and 30 epochs is reasonable.
for epoch in range(100):
    for instance, label in data:
        # Step 1. Remember that Pytorch accumulates gradients.  We need to clear them out
        # before each instance
        model.zero_grad()
    
        # Step 2. Make our BOW vector and also we must wrap the target in a Variable
        # as an integer.  For example, if the target is SPANISH, then we wrap the integer
        # 0.  The loss function then knows that the 0th element of the log probabilities is
        # the log probability corresponding to SPANISH
        bow_vec = autograd.Variable(make_bow_vector(instance, word_to_ix))
        target = autograd.Variable(make_target(label, label_to_ix))
    
        # Step 3. Run our forward pass.
        log_probs = model(bow_vec)
    
        # Step 4. Compute the loss, gradients, and update the parameters by calling
        # optimizer.step()
        loss = loss_function(log_probs, target)
        loss.backward()
        optimizer.step()

In [23]:
for instance, label in test_data:
    bow_vec = autograd.Variable(make_bow_vector(instance, word_to_ix))
    log_probs = model(bow_vec)
    print(log_probs)
print(next(model.parameters())[:,word_to_ix["creo"]]) # Index corresponding to Spanish goes up, English goes down!

Variable containing:
-0.1102 -2.2602
[torch.FloatTensor of size 1x2]

Variable containing:
-2.5811 -0.0787
[torch.FloatTensor of size 1x2]

Variable containing:
 0.5445
-0.4745
[torch.FloatTensor of size 2]



我们得到了正确的答案！可以看到在第一个样本里Spanish的对数概率更加的高，然后在第二个测试样本里English的对数概率更加的高。

# 6. 词嵌入：词汇语义编码
词嵌入是包含每个字典中的词的实值密集向量。在NLP中，词语几乎就是所有的特征!但是怎么样在电脑中表达一个词语？读者可以ASCII码来表达，但是那仅仅只能表达这个词是什么，不能表达其大概的意思（你也许有能力去通过其词缀去获取其词性，或从大小写中获取其性质，但不会太多）。更多地，通过什么样的功能你可以连接这些表达？我们通常希望我们$|V|$维输入的神经网络，其输出密集，其中$V$是我们的词典大小，但是通常输出只有很少的维度（比如如果我们只预测一小量的标签）。那么我们怎么样从一个高维度的空间获取一个较小维度的空间？

替换ASCII表达，我们使用one-hot编码如何呢？这会我们需要表达词语$w$为
$$ \overbrace{\left[ 0, 0, \dots, 1, \dots, 0, 0 \right]}^\text{|V| elements} $$
这里1对于词语$w$来说是一个独占的位置。其它词语也会有独有的一个1的位置，然后其它位置都为0。

这种表达方式有一个巨大的缺点，除了它非常的大型以外，它基本完全独立地处理了所有的词语而缺少了彼此词语间的联系。我们真正想要的是在相似词之间有一些标记。让我们来看一个例子了解为什么？

假设我们构建一个语言模型。假设我们看到了如下的句子
* The mathematician ran to the store.
* The physicist ran to the store.
* The mathematician solved the open problem.

在我们的训练数据里。
现在假设我们得到一个从未在训练数据里见过的句子：
* The physicist solved the open problem.

我们的语言模型可能在这个句子上表现的还不错，但是如果我们考虑到以下两个事实：
* 我们看见"数学家"和"物理学家"在一个句子的同一个位置，它们以某种方式有语义关联。
* 我们看见一个没见过的新句中"数学家"以及现在这个"物理学家"在同一个位置。

然后在没见过的新句子上推断"物理学家"是否会表现更加好？这就是我们提到的相似性的概念：我们指语义相似，而不仅仅是相似的拼写表达。连接稀疏性的语言数据是个一个通过连接我们见过的和没见过的节点的技术。这个课程的例子都依赖于一个基础的语义假设：那些出现在相似文本中的词语彼此之间具有语义上的关联，这被称作[分布假设](https://en.wikipedia.org/wiki/Distributional_semantics)。


### 获取密集词嵌入
我们怎么解决这个问题？怎么样确切地将词语编码成相似的语义？也许我们可以考虑一些语义的特性。举个例子，我们可以发现"数学家"和"物理学家"都会跑步，所以我们也许能给这些词在“可以跑步”的语义特性上一个比较高的评分。考虑到其它一些特性，然后可以想象一下如何为其它的特性评分。

如果每个特性都是一个维度，那么我们可以给每个词语一个向量，就像：
$$ q_\text{mathematician} = \left[ \overbrace{2.3}^\text{can run},
\overbrace{9.4}^\text{likes coffee}, \overbrace{-5.5}^\text{majored in Physics}, \dots \right] $$
$$ q_\text{physicist} = \left[ \overbrace{2.5}^\text{can run},
\overbrace{9.1}^\text{likes coffee}, \overbrace{6.4}^\text{majored in Physics}, \dots \right] $$

然后我们可以通过如下方法获得这些词的相似性度量：
$$ \text{Similarity}(\text{physicist}, \text{mathematician}) = q_\text{physicist} \cdot q_\text{mathematician} $$

通过除以长度进行标准化会更加普遍：
$$ \text{Similarity}(\text{physicist}, \text{mathematician}) = \frac{q_\text{physicist} \cdot q_\text{mathematician}}
{\| q_\text{\physicist} \| \| q_\text{mathematician} \|} = \cos (\phi) $$

这里$\phi$是两个向量的夹角。这即使是，极端相似的两个词（嵌入后的点是同一个方向的）会有相似性1。极端不同的词相似性则为-1。

你可以考虑这部分稀疏的one-hot向量是这些我们定义的新向量的一个特别的案例，它们每个词的相似性都为0，然后我们给每个词一些独有的语义特性。这些新的向量是密集的，也就是说它们整体都是非0的。

但是这些新向量有个很大的问题：你可以连系特定的相关性考虑到成千上万个不同的语义特性，那到底该如何去定义不同特性的数值呢？深度学习的主要思想是用神经网络表达特征，而不是需要程序员去设计它们。所以为什么不让词嵌入作为我们模型的参数，然后通过训练去更新它？这就是我们想做的事。我们需要用神经网络实质上通过学习而获得一些隐藏语义特性。注意到词嵌入很有可能是不被解释的。虽然通过上述的手工向量我们可以看到“数学家”和“物理学家”在都喜欢咖啡上比较相似，但如果我们允许神经网络去学习词嵌入然后发现无论“数学家”还是“科学家”都在第二维度上有着很大的数值，我们并不清楚这是什么意思。也就是在一些隐藏的语义维度上相似，很有可能对我们而言是没有什么合理的解释。

In summary, **word embeddings are a representation of the *semantics* of a word, efficiently encoding semantic information that might be relevant to the task at hand**.  You can embed other things too: part of speech tags, parse trees, anything!  The idea of feature embeddings is central to the field.

总的来说，**词嵌入是一种一个词语语义的表征，有效地编码语义信息很有可能对手边的任务非常重要**。你可以嵌入其它的一些东西，例如词性标签，解析树等等！特征嵌入的思想对这个领域而言非常核心。

### 用Pytorch做词嵌入
在我们实现一个例子和实验前，一些关于怎样使用Pytorch和一般的深度学习程序去进行嵌入的提示。就像制作one-hot向量时去为每个词定义一个独一无二的索引，当制作嵌入时，我们也需要为每个词定义一个索引。这会对检索表而言很关键，嵌入表达被存储在$|V|\times D$的矩阵当中，其中$D$是嵌入表达的维度，以致分配$i$索引的词可以嵌入存储导这个矩阵的第$i$行。在笔者所有的代码中，词的映射至索引都用一个词"word_to_ix"。

这个模块torch.nn.Embedding可以让读者使用词嵌入，需要输入两个参数：字典大小，嵌入表达的维度。

为了能将索引放至表中，读者必须使用torch.LongTensor（因为索引都是整型数据，而不是浮点型）

In [24]:
word_to_ix = { "hello": 0, "world": 1 }
embeds = nn.Embedding(2, 5) # 2 words in vocab, 5 dimensional embeddings
lookup_tensor = torch.LongTensor([word_to_ix["hello"]])
hello_embed = embeds( autograd.Variable(lookup_tensor) )
print(lookup_tensor)
print(hello_embed)


 0
[torch.LongTensor of size 1]

Variable containing:
-0.0714 -1.1325 -0.0699  0.0768 -0.4843
[torch.FloatTensor of size 1x5]



### 示例：N-Gram 语言模型
召回至一个n-gram语言模型中，给定一个包含词语$w$的句子，我们想要计算
$$P(w_i|w_{i-1}, w_{i-2}, \dots, w_{i-n+1})$$
这里$w_i$是这个句子的第$i$个词。

在这个例子当中，我们会计算一些训练样本的损失函数，然后通过反向传播法进行参数更新。

In [25]:
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10
# We will use Shakespeare Sonnet 2
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()
# we should tokenize the input, but we will ignore that for now
# build a list of tuples.  Each tuple is ([ word_i-2, word_i-1 ], target word)
trigrams = [ ([test_sentence[i], test_sentence[i+1]], test_sentence[i+2]) for i in range(len(test_sentence) - 2) ]
print(trigrams[:3]) # print the first 3, just so you can see what they look like

[(['When', 'forty'], 'winters'), (['forty', 'winters'], 'shall'), (['winters', 'shall'], 'besiege')]


In [26]:
vocab = set(test_sentence)
word_to_ix = { word: i for i, word in enumerate(vocab) }

In [27]:
class NGramLanguageModeler(nn.Module):
    
    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)
        
    def forward(self, inputs):
        self.embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(self.embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out)
        return log_probs

In [28]:
losses = []
loss_function = nn.NLLLoss()
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

for epoch in range(10):
    total_loss = torch.Tensor([0])
    for context, target in trigrams:
    
        # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
        # into integer indices and wrap them in variables)
        context_idxs = [ word_to_ix[w] for w in context ]
        context_var = autograd.Variable( torch.LongTensor(context_idxs) )
    
        # Step 2. Recall that torch *accumulates* gradients.  Before passing in a new instance,
        # you need to zero out the gradients from the old instance
        model.zero_grad()
    
        # Step 3. Run the forward pass, getting log probabilities over next words
        log_probs = model(context_var)
    
        # Step 4. Compute your loss function. (Again, Torch wants the target word wrapped in a variable)
        loss = loss_function(log_probs, autograd.Variable(torch.LongTensor([word_to_ix[target]])))
    
        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()
    
        total_loss += loss.data
    losses.append(total_loss/len(trigrams))
print(losses) # The loss decreased every iteration over the training data!

[
 4.6157
[torch.FloatTensor of size 1]
, 
 4.5932
[torch.FloatTensor of size 1]
, 
 4.5708
[torch.FloatTensor of size 1]
, 
 4.5486
[torch.FloatTensor of size 1]
, 
 4.5265
[torch.FloatTensor of size 1]
, 
 4.5046
[torch.FloatTensor of size 1]
, 
 4.4829
[torch.FloatTensor of size 1]
, 
 4.4612
[torch.FloatTensor of size 1]
, 
 4.4397
[torch.FloatTensor of size 1]
, 
 4.4183
[torch.FloatTensor of size 1]
]


### 练习： 词嵌入表达运算之CBOW
CBOW(连续词袋)模型在NLP深度学习中非常频繁的被使用。该模型尝试通过给定少量目标词语前面和后面的文本去预测目标词语。这是一个语言模型的特点，因为CBOW不是一个序列也不会有什么概率性。特别的，CBOW通常用于快速训练词嵌入表达，而这些嵌入表达用于初始化一些更复杂模型的嵌入表达。这通常是一种嵌入表达预训练，这通常能提升个百分之几的性能。

CBOW模型如下，给定一个目标词语$w_i$和一个$N$长度文本框，在$w_{i-1}, \dots, w_{i-N}$ 至 $w_{i+1}, \dots, w_{i+N}$上滑动，所有文本框词语构成的集合为$C$，CBOW试着去最小化
$$ -\log p(w_i | C) = \log \text{Softmax}(A(\sum_{w \in C} q_w) + b) $$
其中$q_w$是词语$w$的词嵌入表达。

用Pytorch，填补下面的类来执行这个模型。一些提示：
* 想一下什么样的参数需要去定义
* 确保所有操作的张量形状正确。如果需要重塑张量，利用 .view()方法。

In [29]:
CONTEXT_SIZE = 2 # 2 words to the left, 2 to the right
raw_text = """We are about to study the idea of a computational process. Computational processes are abstract
beings that inhabit computers. As they evolve, processes manipulate other abstract
things called data. The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells.""".split()
word_to_ix = { word: i for i, word in enumerate(set(raw_text)) }
data = []
for i in range(2, len(raw_text) - 2):
    context = [ raw_text[i-2], raw_text[i-1], raw_text[i+1], raw_text[i+2] ]
    target = raw_text[i]
    data.append( (context, target) )
print(data[:5])
vocab = set(raw_text)
VOCAB_SIZE = len(vocab)
print(VOCAB_SIZE)
word_to_ix = { word: i for i, word in enumerate(vocab) }
EMBEDDING_DIM = 20

[(['We', 'are', 'to', 'study'], 'about'), (['are', 'about', 'study', 'the'], 'to'), (['about', 'to', 'the', 'idea'], 'study'), (['to', 'study', 'idea', 'of'], 'the'), (['study', 'the', 'of', 'a'], 'idea')]
49


In [30]:
class CBOW(nn.Module):
    
    def __init__(self, vocab_size, embedding_dim, context_size):
        super(CBOW, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * 2 * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)
    
    def forward(self, inputs):
        m = inputs.size()[0]
        embeds = self.embeddings(inputs).view((m, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out)
        return log_probs

In [31]:
# create your model and train.  here are some functions to help you make the data ready for use by your module
def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    return idxs

train_x = autograd.Variable(torch.LongTensor([make_context_vector(context[0], word_to_ix) for context in data]))
train_y = autograd.Variable(torch.LongTensor([make_context_vector([context[1]], word_to_ix) for context in data]))

loss_function = nn.NLLLoss()
model = CBOW(VOCAB_SIZE, EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.01)
model.zero_grad()

In [32]:
losses = []
for i in range(10):
    log_probs = model(train_x)
    loss = loss_function(log_probs, train_y.view([-1]))
    loss.backward()
    optimizer.step()
    losses.append(loss.data)
print(losses)

[
 3.9435
[torch.FloatTensor of size 1]
, 
 3.9375
[torch.FloatTensor of size 1]
, 
 3.9255
[torch.FloatTensor of size 1]
, 
 3.9077
[torch.FloatTensor of size 1]
, 
 3.8840
[torch.FloatTensor of size 1]
, 
 3.8545
[torch.FloatTensor of size 1]
, 
 3.8195
[torch.FloatTensor of size 1]
, 
 3.7789
[torch.FloatTensor of size 1]
, 
 3.7329
[torch.FloatTensor of size 1]
, 
 3.6819
[torch.FloatTensor of size 1]
]


# 7. 时序模型与长短期记忆网络(LSTM)
这时，我们已经看过不同的前馈神经网络。这就是说，神经网络并没有保留些什么状态，这应该不是我们想要的行为。时序模型是NLP的核心：这指的是输入需要依赖时间的一些模型。最经典的时序模型案例就是做词性标注的隐马尔科夫模型。另一个例子就是条件随机场。

一个循环神经网络是会维持某种状态的网络。举个例子，它的输出会被用来当作下一时刻的输入，所以信息可以沿着神经网络越过序列时进行传递。一个LSTM的例子就是，对于任意在序列中的元素，都有一个通信的隐藏状态$h_t$，它大体上包括了之前序列中任意点的信息。我们可以在一个语言模型中使用隐状态去预测一个词语，词性标签，以及无数其它玩意儿。

### 用Pytorch处理LSTM
在这个示例前，先注意一些东西。Pytorch的LSTM需要其全部输入是一个三维张量。这些张量轴的语义是很重要的。第一轴是这个序列自身，其第二个轴代表着mini-batch，其第三轴代表着输入的元素。我们还未讨论到mini-batching，所以这里忽略它，并且假设第二轴上仅有1维。如果我们想去在句子"The cow jumped"上去运行这个时序模型，我们的输入看起来应该是
$$ 
\begin{bmatrix}
\overbrace{q_\text{The}}^\text{row vector} \\
q_\text{cow} \\
q_\text{jumped}
\end{bmatrix}
$$
不要忘记这个额外的第二维尺寸是1。

另外，当第一轴的维度是1的时候，读者可以一次只通过这一个序列。

让我们看看快速教程。

In [33]:
lstm = nn.LSTM(3, 3) # Input dim is 3, output dim is 3
torch.manual_seed(1)
inputs = [ autograd.Variable(torch.randn((1,3))) for _ in range(5) ] # make a sequence of length 5
hidden = (autograd.Variable(torch.randn(1,1,3)), autograd.Variable(torch.randn((1,1,3))))
# initialize the hidden state.  
for i in inputs:
    # Step through the sequence one element at a time.
    # after each step, hidden contains the hidden state.
    out, hidden = lstm(i.view(1,1,-1), hidden)
print(out)
print(hidden) 

Variable containing:
(0 ,.,.) = 
 -0.1720  0.3167 -0.2383
[torch.FloatTensor of size 1x1x3]

(Variable containing:
(0 ,.,.) = 
 -0.1720  0.3167 -0.2383
[torch.FloatTensor of size 1x1x3]
, Variable containing:
(0 ,.,.) = 
 -0.4034  0.4309 -0.6180
[torch.FloatTensor of size 1x1x3]
)


In [34]:
# alternatively, we can do the entire sequence all at once.
# the first value returned by LSTM is all of the hidden states throughout the sequence.
# the second is just the most recent hidden state (compare the last slice of "out" with "hidden" below,
# they are the same)
# The reason for this is that:
# "out" will give you access to all hidden states in the sequence
# "hidden" will allow you to continue the sequence and backpropogate, by passing it as an argument
# to the lstm at a later time
torch.manual_seed(1)
inputs = [ autograd.Variable(torch.randn((1,3))) for _ in range(5) ]
hidden = (autograd.Variable(torch.randn(1,1,3)), autograd.Variable(torch.randn((1,1,3))))

inputs = torch.cat(inputs).view(len(inputs), 1, -1) # Add the extra 2nd dimension
out, hidden = lstm(inputs, hidden)
print(out)
print(hidden)

Variable containing:
(0 ,.,.) = 
 -0.3511 -0.1178 -0.5603

(1 ,.,.) = 
 -0.2949 -0.0342 -0.2854

(2 ,.,.) = 
 -0.1284  0.0573 -0.2459

(3 ,.,.) = 
 -0.1964  0.1600 -0.0841

(4 ,.,.) = 
 -0.1720  0.3167 -0.2383
[torch.FloatTensor of size 5x1x3]

(Variable containing:
(0 ,.,.) = 
 -0.1720  0.3167 -0.2383
[torch.FloatTensor of size 1x1x3]
, Variable containing:
(0 ,.,.) = 
 -0.4034  0.4309 -0.6180
[torch.FloatTensor of size 1x1x3]
)


### 示例：一个词性标注的LSTM模型

这块我们会使用LSTM去获得词性标注。我们不使用Viterbi法或者前后向迭代法或者其它类似的算法，但是这里就作为一个给读者的（挑战）练习，思考一下Viterbi怎么运作的可能对接下来做的会有帮助。

这个模型如下：让我们的输入句子为$w_i,\dots,w_M$，其中$w_i \in V$，$V$为我们的词典。还有让$T$作为标签集，$y_i$是词语$w_i$的标注。用$\hat{y}_i$来表示我们对词语$w_i$的预测标签。

这是一个结构体预测模型，我们的输出是的一个序列$\hat{y}_1, \dots, \hat{y}_M$, 其中 $\hat{y}_i \in T$。

为了完成这个预测，将句子上传入LSTM，$h_i$代表$i$时刻的隐藏状态。然后分配每个标签一个独一无二的索引（就好象前面词嵌入表示中使用的word_to_ix）。然后我们对$\hat{y}_i$的预测规则是
$$ \hat{y}_i = \text{argmax}_j \  (\log \text{Softmax}(Ah_i + b))_j $$

这就是说，对隐层状态的线性变化使用对数Softmax，然后将该向量最大的那个值作为预测标签。注意这就意味着目标空间$A$的维度是$|T|$。

In [35]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    tensor = torch.LongTensor(idxs)
    return autograd.Variable(tensor)

In [36]:
training_data = [
    ("The dog ate the apple".split(), ["DET", "NN", "V", "DET", "NN"]),
    ("Everybody read that book".split(), ["NN", "V", "DET", "NN"])
]
word_to_ix = {}
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
print(word_to_ix)
tag_to_ix = {"DET": 0, "NN": 1, "V": 2}

# These will usually be more like 32 or 64 dimensional.
# We will keep them small, so we can see how the weights change as we train.
EMBEDDING_DIM = 6
HIDDEN_DIM = 6

{'The': 0, 'dog': 1, 'ate': 2, 'the': 3, 'apple': 4, 'Everybody': 5, 'read': 6, 'that': 7, 'book': 8}


In [37]:
class LSTMTagger(nn.Module):
    
    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim
        
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        
        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        
        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
        self.hidden = self.init_hidden()
        
    def init_hidden(self):
        # Before we've done anything, we dont have any hidden state.
        # Refer to the Pytorch documentation to see exactly why they have this dimensionality.
        # The axes semantics are (num_layers, minibatch_size, hidden_dim)
        return (autograd.Variable(torch.zeros(1, 1, self.hidden_dim)),
                autograd.Variable(torch.zeros(1, 1, self.hidden_dim)))
        
    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, self.hidden = self.lstm(embeds.view(len(sentence), 1, -1), self.hidden)
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space)
        return tag_scores

In [38]:
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [39]:
# See what the scores are before training
# Note that element i,j of the output is the score for tag j for word i.
inputs = prepare_sequence(training_data[0][0], word_to_ix)
tag_scores = model(inputs)
print(tag_scores)

Variable containing:
-0.8440 -1.2054 -1.3077
-0.8761 -1.1567 -1.3128
-0.8943 -1.1221 -1.3261
-0.8949 -1.1838 -1.2544
-0.9222 -1.2421 -1.1597
[torch.FloatTensor of size 5x3]



In [40]:
for epoch in range(300): # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.  We need to clear them out
        # before each instance
        model.zero_grad()
        
        # Also, we need to clear out the hidden state of the LSTM, detaching it from its
        # history on the last instance.
        model.hidden = model.init_hidden()
    
        # Step 2. Get our inputs ready for the network, that is, turn them into Variables
        # of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)
    
        # Step 3. Run our forward pass.
        tag_scores = model(sentence_in)
    
        # Step 4. Compute the loss, gradients, and update the parameters by calling
        # optimizer.step()
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()

In [41]:
# See what the scores are after training
inputs = prepare_sequence(training_data[0][0], word_to_ix)
tag_scores = model(inputs)
# The sentence is "the dog ate the apple".  i,j corresponds to score for tag j for word i.
# The predicted tag is the maximum scoring tag.
# Here, we can see the predicted sequence below is 0 1 2 0 1
# since 0 is index of the maximum value of row 1,
# 1 is the index of maximum value of row 2, etc.
# Which is DET NOUN VERB DET NOUN, the correct sequence!
print(tag_scores)

Variable containing:
-0.0884 -3.1851 -3.1417
-5.0025 -0.0169 -4.6024
-3.0650 -3.7137 -0.0737
-0.0405 -4.4849 -3.5600
-4.0040 -0.0215 -5.8100
[torch.FloatTensor of size 5x3]



### 练习： 通过字符特征增加LSTM的词性标签
在上述的例子中，每个词拥有一个嵌入式表达，可以充当我们时序模型的输入。让我们通过衍生词语的特征来增大词嵌入的表征。我们期望这应该会起到显著的作用，因为字符级特征信息就像词缀一样在词性上有巨大的轴承作用。举个例子，在英语中带着词缀$-ly$的词通常会被标注为副词。

让$c_w$为词语$w$的字符级特征表达。让$x_w$像之前一样作为词嵌入表达。然后我们时序模型的输入就可以联接$x_w$和$c_w$。那么假如$x_w$有5维而$c_w$有三维，那么我们LSTM就应该接受一个8维的输入。

为了获取字符级表征，完成一次一个单词的字符的LSTM，然后令$c_w$作为LSTM最后的隐层状态。提示：
* 读者新的模型中应该包括两个LSTM。初始的输出词性标注得分（POS tag scores），然后新的则输出一个每个单词的字符级表征。
* 在字符之上完成一个时序模型，你需要去嵌入字符。那字符的嵌入表示就会作为字符LSTM的输入。

# 8. 进阶：动态工具包，动态编程，以及双向长短期记忆-条件随机场（BiLSTM-CRF）

### 动态 vs 静态深度学习工具箱
Pytorch是一个*动态*神经网络工具箱，其它的动态工具箱像是[Dynet](https://github.com/clab/dynet)（笔者注意到这是因为在Pytorch上和Dynet上工作很相似，如果你见过Dynet的例子，它也很有可能对你使用Pytorch有帮助）。对立的就是*静态*工具箱，比如像Theano,Keras,TensorFlow等等。以下是两者的核心区别：
* 在静态工具箱中，一旦定义一个运算图，编译它，然后将实例运作至其中。
* 在动态工具箱中，你需要为*每一个实例*定义运算图。它永远不用被编译也可以即时的被运作。

如果没有经历一些经验，很难去领会区别。一个示例是假设我们想构建一个深度组件解析器，我们的模型大体上包含以下几个部分：
* 我们自下而上的构建了树模型
* 标注了根部节点（句子中的词汇）
* 从这里开始，使用一个神经网络和词嵌入过程去寻找构成组件的组合。无论何时，你都可以形成一个新的组件，使用一些技术去获取一个组件的嵌入式表达。在这个例子中，我们的网络结构是完全依赖于输入句子的。在句子"he green cat scratched the wall"上，在某些模型中，我们想要去联合span$(i,j,r) = (1, 3, \text{NP})$（这就是说，一个NP成分包括了单词1到单词3,在这个例子里就是"The green cat"）。

然而，另外的句子也许会是"Somewhere, the big fat cat scratched the wall"。在这个句子中，我们想在某个点去形成组件$(2,4,\text{NP})$。我们想获取的这些成分依赖于实例。如果我们仅仅一次过在静态工具箱中编译了计算图，那它就会特别不一样或者说是无法将该逻辑编程。但在一个动态工具箱，不存在一次预定义运算图。对每个实例都有新的运算图，所以问题就轻易的解决了。

动态工具箱还有另一个优势就是更容易去debug，而且代码会更加像宿主语言(host language)（这里笔者意思是Pytorch和Dynet看起来比Keras或者Theano更像Python代码）。

笔者这里还提及到差异性，因为在下个区域去执行的模型十分像结构感知器，笔者认为这个模型在静态工具箱上比较难实现。他认为语义结构预测这块是动态工具箱的一个优势。

### BiLSTM-CRF论述
在这个部分，我们会看见完整的使用BiLSTM-CRF去做"命名实体识别"的例子。上述的LSTM标注器对词性标注任务而言是足够的了。但是像CRF这样的时序模型对于"命名实体识别"的强表现是十分重要的，这里假设已经熟悉CRF。虽然这个名字听起来挺吓人的，但这个模型本身就是个CRF，而LSTM只不过用于提供特征。虽然这是个进阶模型，但是远比该教程前期的模型要难的多。如果你已经准备好了，看一下你是否能：
* 写下Viterbi变量第i步递归的第k个标签
* 调整上述递归改为计算前馈变量
* 再调整上述递归去计算对数空间下的前馈变量(提示：log-sum-exp)

如果可以完成上述三件事，你可以去理解下面的代码。回忆CRF计算一个条件概率，让$y$是一个标签句子，$x$是该序列的输入。那么我们需要计算：
$$ P(y|x) = \frac{\exp{(\text{Score}(x, y)})}{\sum_{y'} \exp{(\text{Score}(x, y')})} $$

其中Score是被一些对数势函数$\rm{log} \psi_i(x,y)$定义，如
$$ \text{Score}(x,y) = \sum_i \log \psi_i(x,y) $$
为了更加方便的分割函数，势函数必须看起来只有一个特征。

在BiLSTM-CRF中，我们要定义两种势：”发射“和”转变“。对在索引$i$单词的"发射势"是来自$i$时刻的BiLSTM的隐层状态。"转变势"分值则被存储在了一个大小为$|T|\times|T|$ 的矩阵 $\textbf{P}$里，这里$T$是标签集。在这$\textbf{P}_{i,j}$是指从标签$j$到标签$i$的转变得分。所以：
$$ \text{Score}(x,y) = \sum_i \log \psi_\text{EMIT}(y_i \rightarrow x_i) + \log \psi_\text{TRANS}(y_{i-1} \rightarrow y_i) $$
$$ = \sum_i h_i[y_i] + \textbf{P}_{y_i, y_{i-1}} $$

在这第二个表达式中，我们需考虑到标签是被分配了独一无二的非负指引。
如果上述的探讨太过简短了，可以查看Michael Collins写的[CRFs](http://www.cs.columbia.edu/%7Emcollins/crf.pdf)

### 示例：用BiLSTM-CRF完成命名实体识别

In [42]:
# Helper functions to make the code more readable.
def to_scalar(var):
    # returns a python float
    return var.view(-1).data.tolist()[0]

def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return to_scalar(idx)

# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))
    

class BiLSTM_CRF(nn.Module):
    
    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim):
        super(BiLSTM_CRF, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)
        
        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim//2, num_layers=1, bidirectional=True)
        
        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)
        
        # Matrix of transition parameters.  Entry i,j is the score of transitioning *to* i *from* j.
        self.transitions = nn.Parameter(torch.randn(self.tagset_size, self.tagset_size))
        
        # These two statements enforce the constraint that we never transfer *to* the start tag,
        # and we never transfer *from* the stop tag (the model would probably learn this anyway,
        # so this enforcement is likely unimportant)
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000
        
        self.hidden = self.init_hidden()
        
    def init_hidden(self):
        return ( autograd.Variable( torch.randn(2, 1, self.hidden_dim)),
                 autograd.Variable( torch.randn(2, 1, self.hidden_dim)) )
    
    
    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.Tensor(1, self.tagset_size).fill_(-10000.)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.
        
        # Wrap in a variable so that we will get automatic backprop
        forward_var = autograd.Variable(init_alphas)
        
        # Iterate through the sentence
        for feat in feats:
            alphas_t = [] # The forward variables at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of the previous tag
                emit_score = feat[next_tag].view(1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the edge (i -> next_tag)
                # before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the scores.
                alphas_t.append(log_sum_exp(next_tag_var))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha
        
    def _get_lstm_features(self, sentence):
        self.hidden = self.init_hidden()
        embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)
        lstm_out, self.hidden = self.lstm(embeds)
        lstm_out = lstm_out.view(len(sentence), self.hidden_dim)
        lstm_feats = self.hidden2tag(lstm_out)
        return lstm_feats
        
    def _score_sentence(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = autograd.Variable( torch.Tensor([0]) )
        tags = torch.cat( [torch.LongTensor([self.tag_to_ix[START_TAG]]), tags] )
        for i, feat in enumerate(feats):
            score = score + self.transitions[tags[i+1], tags[i]] + feat[tags[i+1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score
    
    def _viterbi_decode(self, feats):
        backpointers = []
        
        # Initialize the viterbi variables in log space
        init_vvars = torch.Tensor(1, self.tagset_size).fill_(-10000.)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0
        
        # forward_var at step i holds the viterbi variables for step i-1 
        forward_var = autograd.Variable(init_vvars)
        for feat in feats:
            bptrs_t = [] # holds the backpointers for this step
            viterbivars_t = [] # holds the viterbi variables for this step
            
            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the previous step,
                # plus the score of transitioning from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id])
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)
        
        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]
        
        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG] # Sanity check
        best_path.reverse()
        return path_score, best_path
        
    def neg_log_likelihood(self, sentence, tags):
        self.hidden = self.init_hidden()
        feats = self._get_lstm_features(sentence)
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score
        
    def forward(self, sentence): # dont confuse this with _forward_alg above.
        self.hidden = self.init_hidden()
        # Get the emission scores from the BiLSTM
        lstm_feats = self._get_lstm_features(sentence)
        
        # Find the best path, given the features.
        score, tag_seq = self._viterbi_decode(lstm_feats)
        return score, tag_seq


In [43]:
START_TAG = "<START>"
STOP_TAG = "<STOP>"
EMBEDDING_DIM = 5
HIDDEN_DIM = 4

# Make up some training data
training_data = [ (
    "the wall street journal reported today that apple corporation made money".split(),
    "B I I I O O O B I O O".split()
), (
    "georgia tech is a university in georgia".split(),
    "B I O O O O B".split()
) ]

word_to_ix = {}
for sentence, tags in training_data:
    for word in sentence:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
            
tag_to_ix = { "B": 0, "I": 1, "O": 2, START_TAG: 3, STOP_TAG: 4 }

In [44]:
model = BiLSTM_CRF(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM)
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-4)

In [45]:
# Check predictions before training
precheck_sent = prepare_sequence(training_data[0][0], word_to_ix)
precheck_tags = torch.LongTensor([ tag_to_ix[t] for t in training_data[0][1] ])
print(model(precheck_sent))

(Variable containing:
 7.7874
[torch.FloatTensor of size 1]
, [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0])


In [46]:
# Make sure prepare_sequence from earlier in the LSTM section is loaded
for epoch in range(300): # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.  We need to clear them out
        # before each instance
        model.zero_grad()
    
        # Step 2. Get our inputs ready for the network, that is, turn them into Variables
        # of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = torch.LongTensor([ tag_to_ix[t] for t in tags ])
    
        # Step 3. Run our forward pass.
        neg_log_likelihood = model.neg_log_likelihood(sentence_in, targets)
    
        # Step 4. Compute the loss, gradients, and update the parameters by calling
        # optimizer.step()
        neg_log_likelihood.backward()
        optimizer.step()

In [47]:
# Check predictions after training
precheck_sent = prepare_sequence(training_data[0][0], word_to_ix)
print(model(precheck_sent))
# We got it!

(Variable containing:
 21.1687
[torch.FloatTensor of size 1]
, [0, 1, 1, 1, 2, 2, 2, 0, 1, 2, 2])


### 练习：一个用于辨别标签的新损失函数
对我来们来说创建一个运算图去解码其实并不重要，因为这里我们不会在viterbi路径得分中用到向后传播法。尝试训练与Viterbi路径得分和黄金标准路径得分（score of the gold-standard path）不一样损失函数的标注器。这已经很明确该函数是"非负的"，且当为0的时候说明标注序列是正确的。这是很重要的结构感知器。

这个修改会很短，因为Viterbi和score_sentence都已经可以执行了。这是一个依赖训练实例的运算图示例。虽然作者并未尝试用一个静态工具箱去完成它，作者认为这可能会没那么直接。

那么获取一些真实的数据然后做一个对比吧！